# Step 03: Data preparation

In this step we enrich the dataset with some information we can find in the text, like:

* When a person has a special position in the government or in the congress (President of the congress, President of the nation, Minister,...), they usually have the position name written in ALL CAPS and then their name follows it between parens. Like this: `PRESIDENTE DEL GOBIERNO (Mariano Rajoy)`
* There's also a process to enrich the dataset with additional information based on paranthesized text.


In [245]:
import pandas as pd
import re

files = !ls data/debates/*.csv

full = pd.concat([pd.read_csv(f) for f in files],ignore_index=True)

In [246]:
# constants
title_pat=r'(.*)\((.*)\)'

In [247]:
# find patterns like PRESIDENTE DEL GOBIERNO (Mariano Rajoy)

# extract title, ex: "PRESIDENTE DEL GOBIERNO"
full["Title"]=full["Name"].apply(lambda x: re.match(title_pat, x).groups()[0] if re.match(title_pat, x)!=None else None)
# extract name, ex: "Mariano Rajoy"
full["Name"]=full["Name"].apply(lambda x: re.match(title_pat, x).groups()[1].upper() if re.match(title_pat, x)!=None else x)
full[full.Title.notnull()].head()

,Name,Text,Date,Term,Title
37,BELOKI GUERRA,Propuesta de transacción. Extender las relacio...,20080527,IX,SECRETARIO
89,SOLBES MIRA,"Señor presidente, señorías, comparezco de nuev...",20080626,IX,VICEPRESIDENTE SEGUNDO DEL GOBIERNO Y MINISTRO...
102,CUNILLERA I MESTRES,"Muchas gracias, señor Azpiazu. Tiene la palabr...",20080626,IX,VICEPRESIDENTA
104,CUNILLERA I MESTRES,"Señor Sánchez i Llibre, por favor",20080626,IX,VICEPRESIDENTA
106,CUNILLERA I MESTRES,"Gracias a usted, señor Sánchez. Tiene ahora la...",20080626,IX,VICEPRESIDENTA


In [248]:
import datetime
# convert to proper date
full["Date"]=full["Date"].apply(lambda f: str(f))
full.Date=full.Date.apply(lambda d: datetime.date(int(d[0:4]),int(d[4:6]),int(d[6:8])))


In [249]:
# find special one "CANDIDATO" because it is shifted and we have to unshift it
full["temp"]=full[full.Name.str.contains("CANDIDATO")].Name
full.loc[full["temp"].str.contains("CANDIDATO",na=False),"Name"]=full[full.Name.str.contains("CANDIDATO")].Title
full.loc[full["temp"].str.contains("CANDIDATO",na=False),"Title"]=full["temp"]
del full["temp"]
full.head(20)

,Name,Text,Date,Term,Title
0,PRESIDENTE,"Señorías, se inicia la sesión. Tenemos como as...",2008-05-27,IX,None
1,SECO REVILLA,"Sí, prometo. (Aplausos.",2008-05-27,IX,None
2,PRESIDENTE,Don Óscar Seco Revilla ha adquirido la condici...,2008-05-27,IX,None
3,PRESIDENTE,Pasamos a examinar el punto I del orden del dí...,2008-05-27,IX,None
4,LLAMAZARES TRIGO,"Gracias, señor presidente. Señorías, con volun...",2008-05-27,IX,None
5,PRESIDENTE,"Muchas gracias, señor Llamazares. Se ha presen...",2008-05-27,IX,None
6,ESTEBAN BRAVO,"Gracias, señor presidente. El Grupo Vasco enti...",2008-05-27,IX,None
7,PRESIDENTE,"Muchas gracias, señor Esteban. Para fijar posi...",2008-05-27,IX,None
8,JORQUERA CASELAS,"Gracias, señor presidente. Muy brevemente. En ...",2008-05-27,IX,None
9,PRESIDENTE,"Muchas gracias, señor Jorquera. Para fijar la ...",2008-05-27,IX,None


In [250]:
# find paranthesized values and sort them by occurence
parenthesized_pat=r'(\(.+?\))'
# can also use negative matching instead of non-greedy parenthesized_pat=r'.*(\([^\)]+\))'
l=full["Text"].apply(lambda t: ",".join(re.findall(parenthesized_pat,str(t))))
pd.DataFrame({'test':l}).groupby(['test']).size().reset_index().sort_values(0,ascending=False).head(20)

,test,0
0,,229879
17119,(Pausa.),20707
633,(Aplausos),14816
16854,(Pausa),8988
6390,(Convergència i Unió),5055
17469,(Pausa.-Una trabajadora del servicio de limpie...,4559
20516,(Rumores.),3129
19729,(Rumores),2426
7110,(EAJ-PNV),2181
681,"(Aplausos),(Aplausos)",2020


In [251]:
# This is a test to check the paranthesized test it founds
parenthesized_pat=r'\(.+?\)'
unclosed_parenthesized_pat=r'(\(.+?\.)'
m=full["Text"].str.replace(parenthesized_pat,'',regex=True).str.strip()
l=m.apply(lambda t: ",".join([par+")" for par in re.findall(unclosed_parenthesized_pat,str(t))]))
#pd.DataFrame({'test':l}).groupby(['test']).size().reset_index().sort_values(0,ascending=False).head(20)
tdf=pd.DataFrame({'m':l})
tdf["m"].apply(lambda m: m.replace(".)",")") ).iloc[6065]



'(ANTES DENOMINADO PROYECTO DE LEY ORGÁNICA DE MODIFICACIÓN DE LA LEY ORGÁNICA 6/1985, DE 1 DE JULIO, DEL PODER JUDICIAL Y COMPLEMENTARIA A LA LEY PARA LA EJECUCIÓN EN LA UNIÓN EUROPEA DE RESOLUCIONES QUE IMPONGAN SANCIONES PECUNIARIAS),(Número de expediente 121/000003)'

In [252]:
# find paranthesized values and sort them by occurence
parenthesized_pat=r'(\(.+?\))'
unclosed_parenthesized_pat=r'(\(.+?\.)'

t=[]
# can also use negative matching instead of non-greedy parenthesized_pat=r'.*(\([^\)]+\))'
for item in full["Text"]:
    items=re.findall(parenthesized_pat,str(item))
    if(len(items)>0):
        t.append(items)

for item in full["Text"].str.replace(parenthesized_pat,'',regex=True).str.strip():
    items=re.findall(unclosed_parenthesized_pat,str(item))
    if(len(items)>0):
        t.append(items)

flattened_list = [value for sub_list in t for value in sub_list]


In [253]:
parens=pd.DataFrame(flattened_list, columns=["Text"])
parens["Text"]=parens["Text"].str.replace(r"\.$",")")
parens["Text"]=parens["Text"].str.replace(r"\.\)",")")
parens["Text"]=parens["Text"].str.lower()
p=parens.groupby("Text").size().reset_index().sort_values(0,ascending=False)
p[:40]

,Text,0
1367,(aplausos),78403
21555,(pausa),38264
24399,(rumores),25518
3753,(convergència i unió),10246
21661,(pausa.-una trabajadora del servicio de limpie...,5744
4487,(eaj-pnv),4008
24028,(risas),3480
22783,(protestas),2761
8426,(el señor presidente ocupa la presidencia),2017
30794,(votación),1853


In [254]:
# There are two types of parantesized text, the closed and the unclosed.
# We will extract  the closed ones first
parenthesized_pat=r'\(.+?\)'
# And after removing the closed we will seek for unclosed ones
unclosed_parenthesized_pat=r'(\(.+?\.)'

# Add the paranthesized text to the infos
full["infos"]=full["Text"].apply(lambda t: ",".join(re.findall(parenthesized_pat,str(t))).strip())
# now remove parenthesized sentences
full["Text"]=full["Text"].str.replace(parenthesized_pat,'',regex=True).str.strip()
# Add the unclosed sentences
full["infos2"]=full["Text"].apply( lambda t: ",".join([par+")" for par in re.findall(unclosed_parenthesized_pat,str(t))]))
# remove them
full["Text"]=full["Text"].str.replace(unclosed_parenthesized_pat,'',regex=True).str.strip()
# join the two results with comma separated ones into the infos column
full["infos"]=full[["infos","infos2"]].apply( lambda row: row["infos"] if len(row["infos2"])==0 else row["infos2"] if len(row["infos"])==0 else ",".join([row["infos"],row["infos2"]]),axis=1)
full["infos"]=full["infos"].str.replace(".)",")",regex=False)
del full["infos2"]
full["TextLen"]=full.Text.str.len()
full.describe()

,TextLen
count,355529.000000
mean,1470.331863
std,3122.847285
min,0.000000
25%,76.000000
50%,200.000000
75%,1360.000000
max,101263.000000


In [255]:
# extract records
full["Data"]=full["infos"].apply( lambda f: re.search(r"mero de expediente (\d+/\d+)",f).groups()[0] if re.search(r"mero de expediente (\d+/\d+)",f) else None)
full["InterventionType"]=full["infos"].apply( lambda f: "Record" if re.search(r"mero de expediente (\d+/\d+)",f) else None)


In [256]:
# extract congress sentiment from interjections

full["Positive"]=0
full["Negative"]=0
full["Surprise"]=0

positives=[r"aplauso", r"risa", r"muy bien",r"si+!",r"sí",r"asentimiento"]
negatives=[r"rumore",r"no+!",r"protesta",r"denegaciones",r"qué barbaridad",r"mentira"]
surprise=[r"hala",r"oh!",r"ah!"]

for pos in positives:
    full["Positive"]+= full["infos"].apply( lambda f: len(re.findall(pos,f.lower())) if re.search(pos,f.lower()) else 0)

for pos in negatives:
    full["Negative"]+= full["infos"].apply( lambda f: len(re.findall(pos,f.lower())) if re.search(pos,f.lower()) else 0)

for pos in surprise:
    full["Surprise"]+= full["infos"].apply( lambda f: len(re.findall(pos,f.lower())) if re.search(pos,f.lower()) else 0)


full.sort_values("Positive", ascending=False)


,Name,Text,Date,Term,Title,infos,TextLen,Data,InterventionType,Positive,Negative,Surprise
323820,ABASCAL CONDE,"Muchas gracias, señora presidenta. Señorías, b...",2020-10-21,XIV,None,"(aplausos),(Aplausos),(Aplausos),(Aplausos),(A...",101263.0,None,None,105,9,0
281455,SÁNCHEZ PÉREZ-CASTEJÓN,"Señor presidente, señorías, nos encontramos aq...",2016-03-01,XI,CANDIDATO A LA PRESIDENCIA DEL GOBIERNO,"(Aplausos),(Aplausos),(Aplausos),(Protestas),(...",83525.0,None,None,63,46,1
323818,GARRIGA VAZ DE CONCICAO,"Gracias, señora presidenta. Señorías, tengo el...",2020-10-21,XIV,None,"(Aplausos),(Aplausos),(Aplausos),(Aplausos),(r...",56511.0,None,None,50,6,0
221737,RODRÍGUEZ ZAPATERO,"Señor presidente, señorías, quiero que mis pri...",2007-07-03,VIII,PRESIDENTE DEL GOBIERNO,"(Aplausos),(Rumores),(Rumores),(Aplausos),(Apl...",59227.0,None,None,50,14,0
221754,RODRÍGUEZ ZAPATERO,"Muchas gracias, señor presidente. Por eso le d...",2007-07-03,VIII,PRESIDENTE DEL GOBIERNO,"(Aplausos.-Rumores),(Protestas),(El señor Mart...",34169.0,None,None,50,19,1
...,...,...,...,...,...,...,...,...,...,...,...,...
124973,FERNÁNDEZ-MIRANDA Y LOZANA,"Muchas gracias, señor vicepresidente. Para la ...",1999-11-17,VI,VICEPRESIDENTE,,246.0,None,None,0,0,0
124971,FERNÁNDEZ-MIRANDA Y LOZANA,"Señorías, les ruego guarden silencio",1999-11-17,VI,VICEPRESIDENTE,,36.0,None,None,0,0,0
124970,ÁLVAREZ- CASCOS FERNÁNDEZ,"Señor presidente, señorías, el presupuesto de ...",1999-11-17,VI,VICEPRESIDENTE PRIMERO DEL GOBIERNO Y MINISTRO...,"(Feder),(El señor Aguiriano Forniés: Dos años)...",7444.0,None,None,0,0,0
124969,FERNÁNDEZ-MIRANDA Y LOZANA,"Se reanuda la sesión. Corresponde, según el or...",1999-11-17,VI,VICEPRESIDENTE,,378.0,None,None,0,0,0


In [257]:
full.describe()

,TextLen,Positive,Negative,Surprise
count,355529.000000,355626.000000,355626.000000,355626.000000
mean,1470.331863,0.294152,0.124912,0.002041
std,3122.847285,1.056537,0.617666,0.052348
min,0.000000,0.000000,0.000000,0.000000
25%,76.000000,0.000000,0.000000,0.000000
50%,200.000000,0.000000,0.000000,0.000000
75%,1360.000000,0.000000,0.000000,0.000000
max,101263.000000,105.000000,46.000000,7.000000


In [258]:
# Store the full debates to a parquet file
full.to_parquet('../debates.parquet', index=False)

In [259]:
# Find specific politicians and store them as CSV
full[full['Name'].str.contains('EZ ZAPATERO')].to_csv('../zapatero.csv', index=False)
full[full['Name'].str.contains('RAJOY')].to_csv('../rajoy.csv',index=False)

In [260]:
full[full['Name'].str.contains('RAJOY')].iloc[1]["Text"]

'Señor presidente, hace siete días le pregunté aquí sobre la situación de la economía española y las medidas que usted iba a tomar. No me respondió gran cosa, salvo que estábamos preparados mejor que nadie. Yo le voy a dar algunos datos que hemos conocido esta semana, esto es, desde el miércoles pasado: el IPC está en el 4,7 es decir 0,5 puntos más que en abril; el diferencial de inflación con la Unión Europea en el 1,1 es decir 0,2 puntos más que en abril; el dato de paro registrado del mes de mayo es el peor desde 1979; el déficit exterior aumentó en el primer trimestre el 19,1 y hoy es el 11,3 del PIB. Hoy dice la OCDE en sus previsiones que España crecerá el 1,6 este año, menos de la media de la Unión Europea, y hay muchos más datos en estos siete días: sobre ventas minoristas, ventas de automóviles, constituciones de hipotecas, el sector turístico. No voy a entrar en ellos porque no hay tiempo, pero le diré una cosa: la confianza de las empresas y de las familias españolas está en

In [261]:
full[full["Name"].str.contains("RAJOY")].describe()

,TextLen,Positive,Negative,Surprise
count,2123.000000,2123.000000,2123.000000,2123.000000
mean,3701.005652,2.113990,0.740462,0.026849
std,7683.510591,4.321462,1.791697,0.188588
min,2.000000,0.000000,0.000000,0.000000
25%,517.000000,0.000000,0.000000,0.000000
50%,1268.000000,1.000000,0.000000,0.000000
75%,2211.000000,2.000000,1.000000,0.000000
max,74996.000000,48.000000,23.000000,3.000000
